In [1]:
!apt update && apt install -y curl git

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,321 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,664 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,235 kB]
Get:13 http://archive.ubuntu.com

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [11]:
!ollama pull llama3


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a07

In [20]:
!pkill -f ngrok  # Kills all ngrok processes
!fuser -k 8501/tcp  # Kills any process using port 6000 (or use 5000 if needed)
# Install required libraries
OLLAMA_API_URL = "http://127.0.0.1:11434"
!nohup ollama serve > /dev/null 2>&1 &



In [14]:
!pip install streamlit PyPDF2 requests python-docx



In [12]:
!pip install pyngrok

  Using cached pyngrok-7.2.3-py3-none-any.whl.metadata (8.7 kB)
Using cached pyngrok-7.2.3-py3-none-any.whl (23 kB)


In [21]:
from pyngrok import ngrok

# Replace with your ngrok auth token
NGROK_AUTH_TOKEN = "2WG8G2YZNV9jofWBKu5B2E6hA3Q_719vVw9PLU1SwQLwCtcNd"

# Set authentication token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
print("✅ ngrok Auth Token Set!")

public_url = ngrok.connect(8501).public_url
print(f"🚀 Streamlit App is running at: {public_url}")


✅ ngrok Auth Token Set!
🚀 Streamlit App is running at: https://24da-34-143-213-194.ngrok-free.app


In [22]:
%%writefile resume_analyzer.py
import streamlit as st
import PyPDF2
import requests
import tempfile
import os

OLLAMA_URL = "http://127.0.0.1:11434/api/generate"

st.set_page_config(page_title="Resume Analyzer", layout="wide")
st.title("📄 Resume Analyzer - AI Job Compatibility Checker")
st.caption("Upload a resume and enter a job description to analyze compatibility!")

uploaded_file = st.file_uploader("📂 Upload Your Resume (PDF)", type=["pdf"])
job_description = st.text_area("📑 Enter Job Description", height=150)

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() or ""
    return text.strip()

if uploaded_file and job_description:
    st.success(f"✅ Uploaded: {uploaded_file.name}")
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as temp_file:
        temp_file.write(uploaded_file.read())
        temp_pdf_path = temp_file.name

    resume_text = extract_text_from_pdf(temp_pdf_path)
    os.remove(temp_pdf_path)

    if resume_text:
        st.info("📄 Resume extracted successfully!")
        ollama_payload = {
    "model": "llama3",
    "prompt": f"""
    You are an expert HR recruiter analyzing a candidate's resume for job compatibility.

    **Task:** Compare the following resume with the given job description and perform the following analysis:

    **Resume Details:**
    {resume_text}

    **Job Description Details:**
    {job_description}

    **Analysis Criteria:**
    1️⃣ **Experience Check**:
       - Extract the **years of experience required** from the job description.
       - Identify the **candidate’s total years of experience** from the resume.
       - Check if the experience level **matches the requirement**.

    2️⃣ **Skills Match**:
       - Extract the **required technical & soft skills** from the job description.
       - Match them against the skills listed in the resume.
       - Identify **missing or underrepresented skills**.

    3️⃣ **Qualification & Certification Alignment**:
       - Extract the **required qualifications, degrees, and certifications**.
       - Verify if the resume includes them.
       - Highlight **missing qualifications**.

    4️⃣ **Industry & Job Role Fit**:
       - Compare the **industries and job roles** from the resume with the job description.
       - Identify **alignment or mismatches** in domain expertise.

    5️⃣ **Overall Suitability & Score**:
       - Assign a **compatibility score** (High/Medium/Low) with reasoning.
       - Provide **recommendations** for improving the resume to match the job better.

    **Final Output Format:**
    - **Experience Match**: ✅/❌ with explanation
    - **Skills Match**: ✅/❌ with missing skills listed
    - **Qualification Match**: ✅/❌ with missing details
    - **Industry Fit**: ✅/❌ with comments
    - **Compatibility Score**: High/Medium/Low
    - **Suggestions for improvement**: Bullet points

    Please provide a structured response in **clear sections**.
    """,
    "stream": False
}

        response = requests.post(OLLAMA_URL, json=ollama_payload)
        result = response.json().get("response", "⚠️ No response from Llama 3.")
        st.subheader("📌 Compatibility Analysis:")
        st.write(result)


Overwriting resume_analyzer.py


In [ ]:
!streamlit run resume_analyzer.py &





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.143.213.194:8501

